[Macro question (create variables)](https://discourse.julialang.org/t/macro-question-create-variables/90160)

Wanted: a macro

@moo a b  
which is in effect equivalent to

a = foo("a")
b = foo("b")

In [2]:
foo(x) = "$x-$x" 

foo (generic function with 1 method)

In [4]:
macro moo(args...)
    for x in args
        xs = string(x)
        @eval eval(Meta.parse("$($xs) = foo(\"$($xs)\")"))
    end
end

@moo (macro with 1 method)

In [3]:
@moo v1 v2
@show v1 v2

v1 = "v1-v1"
v2 = "v2-v2"


"v2-v2"

In [1]:
using General.Aux

In [4]:
ex = Meta.parse("a")
@logt ex


ex, Symbol
  = a


In [5]:
Meta.parse("a")

:a

In [6]:
dump(ex)

Symbol a


From this symbol, we now want to construct a more complicated expression which contains the literal symbol as well as its name as a string.

In [7]:
res_expr = Meta.parse("a = foo(\"a\")")

:(a = foo("a"))

In [8]:
dump(res_expr)

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol a
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol foo
        2: String "a"


Now, the macro is a pure function which takes the symbol as input and produces the desired expression as output:

In [9]:
macro moo(var)
    # Return expression with symbol interpolated literally on the lhs and as a string on the rhs
    :($(esc(var)) = foo($(string(var))))
end
@macroexpand @moo a

:(a = Main.foo("a"))

Understand macros in general, i.e., not just in Julia, are:

+ [On Lisp](https://www.paulgraham.com/onlisp.html)<br>
    Prentice Hall, 1993, 432 pages, paperback. ISBN 0130305529, incl. code

+ [Practical Common Lisp,](https://gigamonkeys.com/book/) book incl. code

In [15]:
foo(x) = "$x-$x"
macro moo2(args...) 
    v = [:($x = foo($(string(x)))) for x in args]
    return esc(Expr(:block, v...))
end
@moo2 w1 w2
@show w1 w2

w1 = "w1-w1"
w2 = "w2-w2"


"w2-w2"

Now, can anybody explain me why do I need this esc here in the return statement?

This comes down to what we call **“macro hygeine”.** There’s a section in [the documentation here](https://docs.julialang.org/en/v1/manual/metaprogramming/#Hygiene):

In [12]:
macro foo(ex)
    quote
        x = 1
        y = x + $ex
    end
end 

@foo (macro with 1 method)

it would be very bad and surprising if the variables referred to there affected code outside of the macro unless you specifically intend it to. E.g. consider this:

In [13]:
function f(x)
    @foo x
    x
end
f(2)

2

Without macro hygiene, f(2) would return 1, which might be very surprising. So instead what happens is this:

In [14]:
@macroexpand @foo ex

quote
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:3 =#
    var"#91#x" = 1
    #= e:\JuliaProjects\Training3.jl\macro3\31.ipynb:4 =#
    var"#92#y" = var"#91#x" + Main.ex
end

So the intermediate variables here aren’t actually named x or y, they get names that are guaranteed to not clash with surrounding code.

However, sometimes you don’t want this, especialy if you need to actually return and evaluate code that a user passed into the macro. So when you write esc(expr), that essentially is a marker saying "I want everything inside expr to not be made hygenic.

The best practice is to only apply esc to specific parts of the returned expression which need it. So in your case, I would write your macro as

In [18]:
foo(x) = "$x-$x"
macro moo3(args...) 
    v = [:($(esc(x)) = foo($(string(x)))) for x in args]
    return Expr(:block, v...)
end
@moo3 w1 w2

"w2-w2"

because then foo does not get escaped.

Here’s a demonstraction of why:

In [17]:
foo(x) = "$x-$x"
macro moo2(args...) 
    v = [:($x = foo($(string(x)))) for x in args]
    return esc(Expr(:block, v...))
end

let 
    foo(x) = x #some user locally redefines `foo(x)`
    @moo2 w1 w2
    w1, w2
end 

("w1", "w2")

Now look at how @moo3 deals with this:

In [19]:
let 
    foo(x) = x
    @moo3 w1 w2
    w1, w2
end

("w1-w1", "w2-w2")